# Imports

In [2]:
import pandas as pd
import numpy as np
from collections import defaultdict
from sklearn.model_selection import train_test_split

# Data extraction into new Dataframe

In [3]:
# We don't need HomeID and AwayID here, but I put it in just for exploration
results = pd.read_pickle('preprocessed_results.pkl')
df = results[['HomeID', 'AwayID', 'FTHG', 'FTAG', 'FTR']].copy()
df

,HomeID,AwayID,FTHG,FTAG,FTR
0,0,15,3,0,H
1,1,12,1,0,H
2,2,16,0,0,D
3,3,14,6,0,H
4,4,11,2,2,D
...,...,...,...,...,...
13015,97,105,2,2,D
13016,111,106,2,0,H
13017,120,109,0,2,A
13018,122,107,2,2,D


In [4]:
results.sample(3)

,Div,Date,HomeID,AwayID,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR
12539,SP1,2018-12-03,126,117,1,3,A,0,2,A,...,1,4,18,17,6,2,2,3,0,0
3699,D1,2011-04-24,50,36,4,1,H,2,1,H,...,9,4,18,17,7,2,0,0,0,0
2033,E1,2015-11-28,25,17,5,1,H,3,1,H,...,8,2,7,13,7,5,1,1,0,0


In [5]:
# Replace H, D and A by 1, 0 and 2 (target variables)
result_dict = {'H': 1, 'D': 0, 'A': 2}
df['FTR'].replace(result_dict, inplace=True)

In [6]:
#slc = df['AwayID'] == 105
#df[slc][['FTR']].value_counts() / sum(slc)

In [7]:
# The columns H5GR and A5GR contain the points collected by the home
# and away teams over the last 5 games.
# (Win = 3 points, Draw = 1 point, Loss = 0 points)
# If there is not enough data, we just add 1.333333 points (or some mean maybe)
# until we get to 5 games.
df['H5GR'] = 0
df['A5GR'] = 0

cur_game = 0
for game in results.iterrows():      # each game is a tupple of an ID and the actual game information
    team_id = game[1]['HomeID']      # therefore we use game[1]
    team_points = 0         # points from the last 5 games
    
    recent_games = results.loc[(results['HomeID'] == team_id) | (results['AwayID'] == team_id)].tail(6) # TODO Date checken!
    recent_games.drop(recent_games.tail(1).index, inplace=True)   # drop cur_game from recent games
    
    # Add the points accordingly
    for recent in recent_games:
        if recent['FTR'] == 'D':
            team_points += 1
        elif (recent['FTR'] == '1' and recent['HomeID'] == team_id
            or recent['FTR'] == '2' and recent['AwayID'] == team_id):
            team_points += 3           
        # TODO funktioniert noch nicht

    
    df['H5GR'][cur_game] = team_points
    
    cur_game += 1


(0, Div                        E1
Date      2010-08-14 00:00:00
HomeID                      0
AwayID                     15
FTHG                        3
FTAG                        0
FTR                         H
HTHG                        2
HTAG                        0
HTR                         H
HS                         23
AS                         12
HST                        11
AST                         2
HF                         15
AF                         15
HC                         16
AC                          7
HY                          1
AY                          2
HR                          0
AR                          0
Name: 0, dtype: object)


In [ ]:
df.sample(3)